# HuggingFace Data Preparation

This is a notebook to prepare the labelled token dataset for HuggingFace.

## 1. Installs and Imports

In [ ]:
!pip install datasets
!pip install transformers
!pip install s3fs
!pip install boto3
!pip install sagemaker

     |████████████████████████████████| 298 kB 26.5 MB/s 
     |████████████████████████████████| 61 kB 331 kB/s 
     |████████████████████████████████| 243 kB 41.0 MB/s 
     |████████████████████████████████| 1.1 MB 45.9 MB/s 
     |████████████████████████████████| 132 kB 50.8 MB/s 
     |████████████████████████████████| 192 kB 39.3 MB/s 
     |████████████████████████████████| 271 kB 52.6 MB/s 
     |████████████████████████████████| 160 kB 72.2 MB/s 
     |████████████████████████████████| 3.1 MB 37.1 MB/s 
     |████████████████████████████████| 3.3 MB 54.8 MB/s 
     |████████████████████████████████| 895 kB 52.7 MB/s 
     |████████████████████████████████| 596 kB 76.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 8.1 MB 37.8 MB/s 
     |████████████████████████████████| 138

In [ ]:
import os
import json
import s3fs
import boto3
import sagemaker 
import transformers
import pandas as pd
from ast import literal_eval
from datasets import load_dataset
from datasets import Dataset
from datasets import ClassLabel, Sequence
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## 2. Permissions

In [ ]:
system = "COLAB" #["AWS", "COLAB"]

In [ ]:
if system=="AWS":
    fs = s3fs.S3FileSystem()    
    s3_bucket = "govuk-data-infrastructure-integration"
    DATA_DIR = f's3://{s3_bucket}/model-data/govner-data'
    for f in fs.ls(DATA_DIR):
        print(f)
    #Manage interactions with the Amazon SageMaker APIs and any other AWS services needed.
    # sagemaker session bucket -> used for uploading data, models and logs
    # sagemaker will automatically create this bucket if it not exists
    sess = sagemaker.Session() 
    sagemaker_session_bucket= s3_bucket
    if sagemaker_session_bucket is None and sess is not None:
        # set to default bucket if a bucket name is not given
        sagemaker_session_bucket = sess.default_bucket()
        
    role = sagemaker.get_execution_role()
    sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

    print(f"sagemaker role arn: {role}")
    print(f"sagemaker bucket: {sess.default_bucket()}")
    print(f"sagemaker session region: {sess.boto_region_name}")
elif system=="COLAB":
    from google.colab import drive
    drive.mount("/content/gdrive")
    #DATA_DIR = os.path.join("/content/gdrive/My Drive", "NER/Data")
    DATA_DIR = os.path.join("/content/gdrive/Shareddrives/", "GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data")


Mounted at /content/gdrive


In [ ]:
DATA_DIR

'/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data'

## 3. Load Dataset

In [ ]:
file_name = "govuk-labelled-data-ner.csv"

file_path = f"{DATA_DIR}/{file_name}"

print(file_path)

/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/govuk-labelled-data-ner.csv


In [ ]:
label_map_name = "new_label_map.json"

label_map_path = f"{DATA_DIR}/{label_map_name}"

print(label_map_path)

with open(label_map_path) as f:
  label_map = json.load(f)

/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/new_label_map.json


In [ ]:
label_map

{'CONTACT': 1,
 'DATE': 2,
 'EVENT': 3,
 'FINANCE': 4,
 'FORM': 5,
 'LOCATION': 6,
 'MISC': 7,
 'MONEY': 8,
 'O': 0,
 'ORGANIZATION': 9,
 'PERSON': 10,
 'SCHEME': 11,
 'STATE': 12}

In [ ]:
df = pd.read_csv(file_path)

In [ ]:
df.head()

,text,text_token,labels,updated,original_labels,base_path,sampled,label_list,original_file,new_label_list_id
0,"However , to be clear , all of the other check...","['However', ',', 'to', 'be', 'clear', ',', 'al...","[[41, 47, 'EVENT'], [81, 90, 'PERSON']]",NaN,NaN,/government/speeches/business-secretarys-state...,True,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, ..."
1,Organisations that met certain criteria during...,"['Organisations', 'that', 'met', 'certain', 'c...","[[65, 69, 'DATE'], [77, 85, 'FORM']]",NaN,NaN,/guidance/crc-energy-efficiency-scheme-qualifi...,True,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'DATE...",line_by_line_NER_data_sampled_09062020_more_en...,"[0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 5, 0, 0, 0, ..."
2,Use this leaflet and form to find out about pa...,"['Use', 'this', 'leaflet', 'and', 'form', 'to'...","[[21, 25, 'FORM'], [44, 50, 'FINANCE'], [51, 8...",NaN,NaN,/government/publications/social-security-abroa...,True,"['O', 'O', 'O', 'O', 'FORM', 'O', 'O', 'O', 'O...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 4, 4, 4, 0, 6, ..."
3,Legal visits must be booked by email : legalvi...,"['Legal', 'visits', 'must', 'be', 'booked', 'b...","[[31, 36, 'CONTACT'], [82, 94, 'PERSON'], [103...",NaN,NaN,/guidance/liverpool-prison,True,"['O', 'O', 'O', 'O', 'O', 'O', 'CONTACT', 'O',...",line_by_line_NER_data_sampled_12062020_more_en...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 10, 10, 0..."
4,RPC checklist for Small and Micro Business Ass...,"['RPC', 'checklist', 'for', 'Small', 'and', 'M...","[[0, 3, 'ORGANIZATION'], [34, 42, 'ORGANIZATIO...",NaN,NaN,/government/publications/small-and-micro-busin...,True,"['ORGANIZATION', 'O', 'O', 'O', 'O', 'O', 'ORG...",line_by_line_NER_data_sampled_12062020_more_en...,"[9, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 2, 2, 0]"


Evaluate literals

In [ ]:
for col in ['text_token', 'label_list', 'new_label_list_id']:
    print(col)
    df[col] = df[col].map(literal_eval)

text_token
label_list
new_label_list_id


Trim DataFrame to only the useful columns.

In [ ]:
df_trim = df[['text_token', 'new_label_list_id']]

In [ ]:
hf_dataset = Dataset.from_pandas(df_trim)

In [ ]:
hf_dataset

Dataset({
    features: ['text_token', 'new_label_list_id'],
    num_rows: 347216
})

## 4. Dataset Exploration

In [ ]:
print(hf_dataset['text_token'][9])
print(hf_dataset['new_label_list_id'][9])
# print(hf_dataset['new_label_list_id'][9])

['New', 'Style', 'JSA', 'is', 'a', 'fortnightly', 'payment', 'that', 'can', 'be', 'claimed', 'on', 'its', 'own', 'or', 'at', 'the', 'same', 'time', 'as', 'Universal', 'Credit', '.']
[0, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 4, 4, 0]


In [ ]:
for j in ['text_token', 'new_label_list_id']:
  print("{}: {}".format(j, hf_dataset.features[f"{j}"]))
print()

text_token: Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
new_label_list_id: Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)



In [ ]:
labels = [i for i in label_map.keys()]
print(len(labels))
print(labels)

13
['O', 'CONTACT', 'DATE', 'EVENT', 'FINANCE', 'FORM', 'LOCATION', 'MISC', 'MONEY', 'ORGANIZATION', 'PERSON', 'SCHEME', 'STATE']


In [ ]:
hf_dataset.features['new_label_list_id'] = Sequence(ClassLabel(13, labels), -1 , id=None)

In [ ]:
hf_dataset.features['new_label_list_id']

Sequence(feature=ClassLabel(num_classes=13, names=['O', 'CONTACT', 'DATE', 'EVENT', 'FINANCE', 'FORM', 'LOCATION', 'MISC', 'MONEY', 'ORGANIZATION', 'PERSON', 'SCHEME', 'STATE'], names_file=None, id=None), length=-1, id=None)

In [ ]:
label_list = hf_dataset.features["new_label_list_id"].feature.names
label_list

['O',
 'CONTACT',
 'DATE',
 'EVENT',
 'FINANCE',
 'FORM',
 'LOCATION',
 'MISC',
 'MONEY',
 'ORGANIZATION',
 'PERSON',
 'SCHEME',
 'STATE']

## 8. Train/Eval/Test Splits

We must split the data into Training, Evaluation and Test splits.

CONLL Dataset Has the following spits:
* Training: 14,041
* Evaluation: 3,250
* Test: 3,454

In [ ]:
conll_training = {'name':'training', 'total':14041}
conll_evaluation = {'name':'evaluation', 'total':3250}
conll_test = {'name':'test', 'total':3454}

total = conll_training['total'] + conll_evaluation['total'] + conll_test['total']
total

20745

In [ ]:
for i in [conll_training, conll_evaluation, conll_test]:
    i['proportion'] = (i['total'] / total) * 100
    print(i['name'], i['proportion'])

training 67.68377922390938
evaluation 15.66642564473367
test 16.649795131356953


In [ ]:
hf_dataset

Dataset({
    features: ['text_token', 'new_label_list_id'],
    num_rows: 347216
})

In [ ]:
hf_dataset = hf_dataset.train_test_split(train_size=0.85, seed=42)

In [ ]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['text_token', 'new_label_list_id'],
        num_rows: 295133
    })
    test: Dataset({
        features: ['text_token', 'new_label_list_id'],
        num_rows: 52083
    })
})

Add validation split.

In [ ]:
# hf_dataset_clean = hf_dataset["train"].train_test_split(train_size=0.8, seed=42)
# # Rename the default "test" split to "validation"
# hf_dataset_clean["validation"] = hf_dataset_clean.pop("test")
# # Add the "test" set to our `DatasetDict`
# hf_dataset_clean["test"] = hf_dataset["test"]

In [ ]:
# hf_dataset_clean

## 9. Upload splits to gdrive

After we processed the datasets we are going to upload our dataset to gdrive.

In [ ]:
dataset_name = "hf_govuk_data"

In [ ]:
dataset_name_path = f'{DATA_DIR}/{dataset_name}'
dataset_name_path

'/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/hf_govuk_data'

In [ ]:
# save train_dataset to gdrive
hf_input_path = f'{dataset_name_path}'
hf_dataset.save_to_disk(hf_input_path)

Flattening the indices:   0%|          | 0/296 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/53 [00:00<?, ?ba/s]

## 10. Download Splits

In [ ]:
from datasets import load_dataset, load_metric, load_from_disk

hf_data = 'hf_govuk_data'

hf_data_path = f'{DATA_DIR}/{hf_data}'
hf_data_path

'/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/hf_govuk_data'

In [ ]:
datasets = load_from_disk(hf_data_path)

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['new_label_list_id', 'text_token'],
        num_rows: 295133
    })
    test: Dataset({
        features: ['new_label_list_id', 'text_token'],
        num_rows: 52083
    })
})

In [ ]:
datasets['train'].features

{'new_label_list_id': Sequence(feature=ClassLabel(num_classes=13, names=['O', 'CONTACT', 'DATE', 'EVENT', 'FINANCE', 'FORM', 'LOCATION', 'MISC', 'MONEY', 'ORGANIZATION', 'PERSON', 'SCHEME', 'STATE'], names_file=None, id=None), length=-1, id=None),
 'text_token': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

## 11. Make smaller sampled dataset

In [ ]:
df_trim = df[['text_token', 'new_label_list_id']]

In [ ]:
samp_df_trim = df_trim.sample(10000).reset_index()
samp_df_trim = samp_df_trim[['text_token', 'new_label_list_id']]
samp_df_trim.shape

(10000, 2)

In [ ]:
samp_df_trim.head()

,text_token,new_label_list_id
0,"[The, newspaper, did, not, say, how, it, obtai...","[0, 9, 0, 0, 0, 0, 0, 0, 0, 7, 0]"
1,"[This, is, necessary, for, wider, prescribing,...","[0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 2, 0]"
2,"[PDF, ,, 330KB, ,, 4, pages, Overview, 3, :, C...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[All, travellers, coming, from, endemic, count...","[0, 10, 0, 0, 0, 6, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0]"
4,"[Children, ’, s, visits, are, where, children,...","[10, 0, 0, 0, 0, 0, 10, 0, 0, 10, 0, 3, 0, 0, ..."


In [ ]:
samp_hf_dataset = Dataset.from_pandas(samp_df_trim)

In [ ]:
samp_hf_dataset

Dataset({
    features: ['text_token', 'new_label_list_id'],
    num_rows: 10000
})

In [ ]:
samp_hf_dataset[0]

{'new_label_list_id': [0, 9, 0, 0, 0, 0, 0, 0, 0, 7, 0],
 'text_token': ['The',
  'newspaper',
  'did',
  'not',
  'say',
  'how',
  'it',
  'obtained',
  'the',
  'photos',
  '.']}

In [ ]:
samp_hf_dataset.features['new_label_list_id'] = Sequence(ClassLabel(13, labels), -1 , id=None)

In [ ]:
samp_hf_dataset.features['new_label_list_id']

Sequence(feature=ClassLabel(num_classes=13, names=['O', 'CONTACT', 'DATE', 'EVENT', 'FINANCE', 'FORM', 'LOCATION', 'MISC', 'MONEY', 'ORGANIZATION', 'PERSON', 'SCHEME', 'STATE'], names_file=None, id=None), length=-1, id=None)

In [ ]:
label_list = samp_hf_dataset.features["new_label_list_id"].feature.names
label_list

['O',
 'CONTACT',
 'DATE',
 'EVENT',
 'FINANCE',
 'FORM',
 'LOCATION',
 'MISC',
 'MONEY',
 'ORGANIZATION',
 'PERSON',
 'SCHEME',
 'STATE']

In [ ]:
samp_hf_dataset = samp_hf_dataset.train_test_split(train_size=0.85, seed=42)

In [ ]:
samp_hf_dataset

DatasetDict({
    train: Dataset({
        features: ['text_token', 'new_label_list_id'],
        num_rows: 8500
    })
    test: Dataset({
        features: ['text_token', 'new_label_list_id'],
        num_rows: 1500
    })
})

After we processed the datasets we are going to upload our dataset to gdrive.

In [ ]:
samp_dataset_name = "samp_hf_govuk_data"

In [ ]:
samp_dataset_path = f'{DATA_DIR}/{samp_dataset_name}'
samp_dataset_path

'/content/gdrive/Shareddrives/GOV.UK teams/2020-2021/Data labs/content-metadata-2021/Data/samp_hf_govuk_data'

In [ ]:
# save train_dataset to gdrive
samp_hf_input_path = f'{samp_dataset_path}'
samp_hf_dataset.save_to_disk(samp_hf_input_path)

Flattening the indices:   0%|          | 0/9 [00:00<?, ?ba/s]

Flattening the indices:   0%|          | 0/2 [00:00<?, ?ba/s]